In [261]:
import pandas as pd
import numpy as np

import json
import datetime

import datetime
import requests

import os

from datetime import timedelta

from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go

import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
import matplotlib
plt.style.use('ggplot')
from matplotlib.pyplot import figure

%matplotlib inline
matplotlib.rcParams['figure.figsize'] = (12,8)

from sklearn.tree import DecisionTreeRegressor
from sklearn import tree
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, GradientBoostingClassifier
import graphviz
from sklearn.model_selection import cross_val_score
from sklearn.ensemble.partial_dependence import plot_partial_dependence, partial_dependence
from mpl_toolkits.mplot3d import Axes3D
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error

init_notebook_mode(connected=True)
pd.options.mode.chained_assignment = None

import math

In [3]:
df = pd.read_csv('~/Downloads/^GSPC.csv')
df_bond = pd.read_csv('~/Downloads/VBMFX.csv')

In [4]:
df['Date'] = pd.to_datetime(df['Date'])
df['year'] = df['Date'].dt.year

df_bond['Date'] = pd.to_datetime(df_bond['Date'])
df_bond['year'] = df_bond['Date'].dt.year

In [ ]:
trace = go.Scatter(
    x = df['Date'],
    y = df['Adj Close'],
    mode = 'lines'
)

iplot([trace])

In [6]:
df['last_day_return'] = df['Adj Close']/df['Adj Close'].shift(1) - 1
df_bond['last_day_return'] = df_bond['Adj Close']/df_bond['Adj Close'].shift(1) - 1

In [7]:
df['Date'].describe()

count                   23084
unique                  23084
top       1967-09-20 00:00:00
freq                        1
first     1927-12-30 00:00:00
last      2019-11-22 00:00:00
Name: Date, dtype: object

In [8]:
msk = (df['Date'] >= '1986-12-11') & (df['Date'] <= '2019-11-22')
df.loc[msk, 'last_day_return'].describe()

count    8306.000000
mean        0.000367
std         0.011299
min        -0.204669
25%        -0.004328
50%         0.000582
75%         0.005596
max         0.115800
Name: last_day_return, dtype: float64

In [9]:
msk = (df_bond['Date'] >= '1986-12-11') & (df_bond['Date'] <= '2019-11-22')
df_bond.loc[msk, 'last_day_return'].describe()

count    8305.000000
mean        0.000230
std         0.002708
min        -0.016598
25%        -0.001052
50%         0.000000
75%         0.001861
max         0.042238
Name: last_day_return, dtype: float64

In [ ]:
data = [go.Histogram(x=df['last_day_return'], nbinsx=1000), go.Histogram(x=df_bond['last_day_return'], nbinsx=1000)]
iplot(data)

In [489]:
%%time
msk = (df['Date'] >= '1986-12-11') & (df['Date'] <= '2019-11-22')
df_sim = df[msk]

msk = (df_bond['Date'] >= '1986-12-11') & (df_bond['Date'] <= '2019-11-22')
df_sim_bond = df_bond[msk]

chunk_years = [15, 15, 15, 15, 5]

percent_bond = 0.5

num_trials = 2000
start_p = 1000000

inflation = 0.03
initial_monthly_withdrawal = 15250

p_list = []
w_list = []

for trial in range(num_trials):
    portfolio_bonds = start_p*percent_bond
    portfolio_stocks = start_p - portfolio_bonds
    monthly_withdrawal = initial_monthly_withdrawal
    total_withdrawals = 0
    
    for chunk in chunk_years:
        # randomly choose a one-year stock market period.
        start_ind = np.random.choice(range(len(df_sim.index) - 252*chunk))
        end_ind = start_ind + 252*chunk
        
        # get the returns for the period.
        ret_stocks = (df_sim.iloc[start_ind:end_ind]['last_day_return']) + 1
        ret_bonds = (df_sim_bond.iloc[start_ind:end_ind]['last_day_return']) + 1
        
        for month in range(12*chunk):
            
            if (month % 12 == 0) and (month > 0):
                # adjust monthly withdrawal by inflation for the next year.
                monthly_withdrawal *= 1+inflation
                
            if (portfolio_stocks + portfolio_bonds) <= 0:
                break
            
            # withdraw amount.
            stock_withdrawal = monthly_withdrawal*(1-percent_bond)
            bond_withdrawal = monthly_withdrawal - stock_withdrawal
            withdraw_amt = 0
            
            if portfolio_stocks > stock_withdrawal:
                # have enough money to withdraw from stocks.
                portfolio_stocks -= stock_withdrawal
                withdraw_amt += stock_withdrawal
            else:
                # not enough in the stock portfolio. But still can withdraw it all.
                stock_withdrawal -= portfolio_stocks
                withdraw_amt += portfolio_stocks
                
                # withdraw the remaining amount from the bond portfolio (if able to).
                stock_withdraw_remaining = stock_withdrawal - portfolio_stocks
                if portfolio_bonds > stock_withdraw_remaining:
                    # enough money in bonds to make the withdrawal.
                    portfolio_bonds -= stock_withdraw_remaining
                    withdraw_amt += stock_withdraw_remaining
                else:
                    # not enough money in bonds to make the full withdrawal. Failure!
                    withdraw_amt += portfolio_bonds
                    portfolio_bonds = 0
            
                # stock portfolio is now empty.
                portfolio_stocks = 0

            if portfolio_bonds > bond_withdrawal:
                # have enough money to withdraw from bonds.
                portfolio_bonds -= bond_withdrawal
                withdraw_amt += bond_withdrawal
            else:
                # not enough in the bond portfolio. But still can withdraw it all.
                bond_withdrawal -= portfolio_bonds
                withdraw_amt += portfolio_bonds
                
                # withdraw the remaining amount from the stock portfolio (if able to).
                bond_withdraw_remaining = bond_withdrawal - portfolio_bonds
                if portfolio_stocks > bond_withdraw_remaining:
                    # enough money in stocks to make the withdrawal.
                    portfolio_stocks -= bond_withdraw_remaining
                    withdraw_amt += bond_withdraw_remaining
                else:
                    # not enough money in stocks to make the full withdrawal. Failure!
                    withdraw_amt += portfolio_stocks
                    portfolio_stocks = 0
                    
                # stock portfolio is now empty.
                portfolio_bonds = 0
            
            # rebalance portfolio.
            total_portfolio = portfolio_stocks + portfolio_bonds
            portfolio_bonds = total_portfolio*percent_bond
            portfolio_stocks = total_portfolio - portfolio_bonds
            
            # return from portfolio.
            ind1 = month * 21
            ind2 = ind1 + 21
            if portfolio_stocks > 0:
                # calculate the return on investment (stocks) for the month.
                month_ret_stocks = np.prod(ret_stocks[ind1:ind2])
                portfolio_stocks *= month_ret_stocks
                
            if portfolio_bonds > 0:
                # calculate the return on investment (bonds) for the month.
                month_ret_bonds = np.prod(ret_bonds[ind1:ind2])
                portfolio_bonds *= month_ret_bonds
            
            if trial == 1:
                print('{}, {}, monthly_withdrawal: {}, withdrawal amt: {}, portfolio_stocks: {}, portfolio_bonds: {}, stock_return: {}, bond_return: {}'.format(chunk, month, monthly_withdrawal, withdraw_amt, portfolio_stocks, portfolio_bonds, round(month_ret_stocks,5), round(month_ret_bonds,5)))
            
            total_withdrawals += withdraw_amt
        
        
        # stop the trial if there's no more money.
        if (portfolio_stocks + portfolio_bonds) <= 0:
            break
            
        monthly_withdrawal *= 1+inflation
            
    w_list.append(total_withdrawals)
    p_list.append(portfolio_stocks + portfolio_bonds)


5, 0, monthly_withdrawal: 15250, withdrawal amt: 15250.0, portfolio_stocks: 459782.3780076583, portfolio_bonds: 497560.086058862, stock_return: 0.93381, bond_return: 1.01053
5, 1, monthly_withdrawal: 15250, withdrawal amt: 15250.0, portfolio_stocks: 439282.71550191956, portfolio_bonds: 474896.11954976915, stock_return: 0.93257, bond_return: 1.00817
5, 2, monthly_withdrawal: 15250, withdrawal amt: 15250.0, portfolio_stocks: 469368.02749274665, portfolio_bonds: 451029.7432589988, stock_return: 1.04428, bond_return: 1.00348
5, 3, monthly_withdrawal: 15250, withdrawal amt: 15250.0, portfolio_stocks: 468473.2850031724, portfolio_bonds: 451365.5679493727, stock_return: 1.03513, bond_return: 0.99733
5, 4, monthly_withdrawal: 15250, withdrawal amt: 15250.0, portfolio_stocks: 430658.16652633715, portfolio_bonds: 460670.5576903034, stock_return: 0.95216, bond_return: 1.01852
5, 5, monthly_withdrawal: 15250, withdrawal amt: 15250.0, portfolio_stocks: 433060.512495876, portfolio_bonds: 441664.4400

In [490]:
portfolio_s = pd.Series(p_list)
success_rate = np.sum(portfolio_s > 0)/num_trials*100
print(f'success rate: {success_rate}%')

success rate: 98.75%


In [466]:
pd.Series(w_list).describe()

count      2000.000000
mean     969464.899104
std        1605.480881
min      946630.642483
25%      969657.843877
50%      969657.843877
75%      969657.843877
max      969657.843877
dtype: float64